In [1]:
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import yaml
from icecream import ic
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import matplotlib.pyplot as plt
import random


In [2]:
#HW: 
#1) better looking code
#2) working program
#3) data generator(pytorch Dataset, DataLoader):
#     sinusoids with different frequentions ~ U(1,10) - generate from 0 to 1000 (step = 1/0.1/0.01) 
#4) create NN predicting frequentions (regression) 
#5) plot graph

In [3]:
torch.manual_seed(22)

In [4]:
# TODO try primitive convoltional and inceptiontime
# TODO comment everything 


In [5]:
DEVICE = 'cuda'
writer = SummaryWriter()

In [6]:
class SineDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data_count, device='cpu'):
        self.freq = 10*torch.rand(size=(data_count, 1), device=DEVICE) + 1
        
        random_uniform_shift = 10*torch.rand(size=(data_count, 1), device=DEVICE)
        random_normal_shift = torch.normal(0, 1, size=(1, 1), device=DEVICE)
        
        self.points = torch.arange(0,6,1/40, device=DEVICE).repeat(data_count, 1)  + random_uniform_shift
        self.phase = torch.normal(0, 5, size=(data_count, 1), device=DEVICE)
        self.amplitude = (self.points[0] - random_uniform_shift[0])**2 
        self.data_matrix = torch.sin(self.points * self.freq + self.phase)
        
        for i in range(data_count):
            if random.random() < 0.3:
                self.amplitude = torch.flip(self.amplitude, dims=(-1,))
                random_noise = (self.amplitude+1)*torch.normal(0, 1, size=(1, len(self.data_matrix[0])), device=DEVICE)
                self.data_matrix[i] = self.amplitude*self.data_matrix[i] + random_noise
            elif 0.3<=random.random() <0.7:
                random_noise = (self.amplitude+1)*torch.normal(0, 1, size=(1, len(self.data_matrix[0])), device=DEVICE)
                self.data_matrix[i] = self.amplitude*self.data_matrix[i] + random_noise               
            else:
                random_noise = torch.normal(0, 1, size=(1, len(self.data_matrix[0])), device=DEVICE)
                self.data_matrix[i] = self.data_matrix[i] + random_noise
                
    def __len__(self):
        return len(self.freq)

    def __getitem__(self, idx):
        return self.data_matrix[idx], self.freq[idx]


In [7]:
class NN(nn.Module):
    def __init__(self, layers: list[dict]):
        super().__init__()
        self.layers = nn.ModuleList(layers)

    def forward(self,x):
        for layer in self.layers:
            x = F.relu(layer(x))
        return x

In [8]:
class CNN(nn.Module):
    def __init__(self, conv_layers: list[dict], fullycon_layers: list [dict]):  # todo: repair
        super().__init__()
        self.conv_layers = nn.ModuleList(conv_layers)
        self.fullycon_layers = nn.ModuleList(fullycon_layers)
        
    def forward(self,x):
        x = torch.unsqueeze(x, 1)
        for layer in self.conv_layers:
            x = F.relu(layer(x))
            
        x = torch.flatten(x, -1)
        
        for layer in self.fullycon_layers:
            x = F.relu(layer(x))
        print(x.shape)   
        return x

In [17]:
class NeuroNet:
    def __init__(self, control_center: str, meta: dict = ''):
        self._load_yaml(control_center)
        self.model = self.build_model()
        self.model.to('cuda')
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, self.training_params["epoch_num"])
        self.criterion = nn.MSELoss()
        self.history = []
    def _load_yaml(self, yaml_path:Path) -> None:
        
        path = Path(yaml_path) 
        with path.open(mode = "r") as yaml_file:
            data = yaml.load(yaml_file, Loader = yaml.SafeLoader)
            
         
        self.training_params=data["training_params"]
        self.eval_params=data["eval_params"]
        self.lr = data["lr"]
        
  
    # load convolutional layers
        self.conv_layers_config = data['conv_layers']
        self.conv_layers = []
        for layer_config in self.conv_layers_config:
            self.conv_layers.append(eval('nn.' + layer_config['name'])(*layer_config.get('args',[]), **layer_config.get('kwargs', {})))
    # load fully connected layers    
        self.fullycon_layers_config = data['fullycon_layers']
        self.fullycon_layers = []
        for layer_config in self.fullycon_layers_config:
            self.fullycon_layers.append(eval('nn.' + layer_config['name'])(*layer_config.get('args',[]), **layer_config.get('kwargs', {})))
            
    def build_model(self):
        return NN(self.fullycon_layers) 
        
    def train_model(self, training_data: Dataset, testing_data: Dataset):

        step = 0

        train_dataloader = DataLoader(training_data, **self.training_params.get("dataloader_params", {}))
        test_dataloader = DataLoader(testing_data, **self.eval_params)

        for epoch in range(self.training_params["epoch_num"]):
            self.model.train()

            for batch_id, (inputs, targets) in enumerate(
                    tqdm(train_dataloader, desc=f'epoch {epoch + 1}/ {self.training_params["epoch_num"]} ')):
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)
                loss.backward()
                self.optimizer.step()
                writer.add_scalar('Training Loss', loss, global_step=step)
                step += 1

            self.model.eval()  # TODO make validation loss
            for batch_id, (inputs, targets) in enumerate(test_dataloader):
                outputs = self.model(inputs)
                mse = np.mean((outputs.detach().cpu().numpy() - targets.detach().cpu().numpy()) ** 2)
                # print(mse)
                writer.add_scalar('mean square error', mse, global_step=epoch)
                self.history.append(mse)

            ic(self.scheduler.get_last_lr())
            last_lr = self.scheduler.get_last_lr()[0]  # WHY IS IT A LIST
            writer.add_scalar('learning rate', last_lr, global_step=epoch)
            self.scheduler.step()
        writer.close()
        
    def predict(self, points: torch.Tensor):
        return self.model(points)

In [14]:
dataset = SineDataset(128*128, DEVICE)
train_data, test_data = random_split(dataset, [100*128,28*128])

In [18]:
neuro_net= NeuroNet('MLP.yaml')

KeyError: 'conv_layers'

In [19]:
neuro_net.train_model(train_data,test_data)

epoch 1/ 10 :   0%|                                                                                                                                                                                                                                               | 0/200 [00:00<?, ?it/s]

torch.Size([64, 8, 1])


RuntimeError: The size of tensor a (8) must match the size of tensor b (64) at non-singleton dimension 1

In [ ]:
print(f"mean square error: {neuro_net.history[-1]}")

In [ ]:
plt.plot(neuro_net.history)
plt.show()

In [ ]:
for i in range(10):
    points, freq = test_data[i]
    prediction = neuro_net.predict(points)
    print(f'Actuall freq: {freq}, Predicted freq: {prediction}') 

In [ ]:
for i in range(10):
    plt.plot(dataset.points[i].cpu(), dataset.data_matrix[i].cpu())
    # TODO original sine waves without noise with 
    plt.show()    

TODO:
- Validation loss
- random shifts
- random noise(rand float*constant), plot graph (constant/validation_loss)
- random aplitudes(rand int)
- 